In [4]:
import pandas as pd
import quandl
import math

In [7]:
df = quandl.get('WIKI/GOOGL')
df = df[['Adj. Open','Adj. High', 'Adj. Low', 'Adj. Close','Adj. Volume']]
df['PCT'] = (df['Adj. Close']-df['Adj. Open'])/df['Adj. Open'] * 100
df

LimitExceededError: (Status 429) (Quandl Error QELx01) You have exceeded the anonymous user limit of 50 calls per day. To make more calls today, please register for a free Quandl account and then include your API key with your requests.

In [ ]:
df['HL'] = (df['Adj. High']-df['Adj. Low'])/df['Adj. Low'] *100
df = df[['Adj. Close','HL','PCT','Adj. Volume']]
#df.fillna(-9999, inplace = True)
df.dropna(inplace = True)
forcast_col = 'Adj. Close'
forcast_out = int(math.ceil(0.01*len(df)))
# print(len(df))

In [ ]:
df['label'] = df[forcast_col].shift(-forcast_out)
#df.dropna(inplace = True)
df

In [ ]:
import numpy as np
from sklearn import preprocessing, svm
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
import pickle

In [ ]:
my_x = np.array(df.drop(['label'],1))[-forcast_out:]
df.dropna(inplace = True)
x = np.array(df.drop(['label'],1))
y = np.array(df['label'])
#x = preprocessing.scale(x)
print(len(my_x))
print(len(x),len(y))
#df.dropna(inplace = True)
df

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size = 0.2)
classifier = LinearRegression() #svm.SVR(kernel = 'poly')
classifier.fit(x_train, y_train)
with open ('linear_reg.pickle', 'wb') as f:
    pickle.dump(classifier, f)
pickle_in = open('linear_reg.pickle', 'rb')
classifier = pickle.load(pickle_in)
accuracy = classifier.score(x_test, y_test)
forcast = classifier.predict(my_x)
print(forcast, accuracy, forcast_out)

In [ ]:
import datetime
import matplotlib.pyplot as plt
from matplotlib import style
style.use('ggplot')

In [ ]:
df['forcast'] = np.nan
last_date = df.iloc[-1].name
last_unix = last_date.timestamp()
one_day = 86400
next_unix = last_unix + one_day
print(datetime.datetime.fromtimestamp(next_unix))

In [ ]:
for i in forcast:
    next_date = datetime.datetime.fromtimestamp(next_unix)
    next_unix += one_day
    df.loc[next_date] = [np.nan for _ in range(len(df.columns)-1)]+[i]

In [ ]:
df['Adj. Close'].plot()
df['forcast'].plot()
plt.legend(loc = 4)
plt.xlabel('Date')
plt.ylabel('Price')
plt.show()